In [2]:
import gym
import pandas as pd
import numpy as np
from stable_baselines3 import QLearningAgent

ImportError: cannot import name 'QLearningAgent' from 'stable_baselines3' (/home/pisky/PyCharm enviroments/mem-ind/lib/python3.10/site-packages/stable_baselines3/__init__.py)

In [1]:
def calculate_rsi(prices, n=30):
    deltas = np.diff(prices)
    seed = deltas[:n + 1]
    up = seed[seed >= 0].sum() / n
    down = -seed[seed < 0].sum() / n
    rs = up / down
    rsi = [np.nan] * (n - 1) + [100. - 100. / (1. + rs)]

    for i in range(n, len(prices)):
        delta = deltas[i - 1]  # Cambio actual
        if delta > 0:
            upval = delta
            downval = 0.
        else:
            upval = 0.
            downval = -delta
        up = (up * (n - 1) + upval) / n
        down = (down * (n - 1) + downval) / n
        rs = up / down
        rsi.append(100. - 100. / (1. + rs))

    return rsi

In [ ]:

import numpy as np

class RSIEnv(gym.Env):
    def __init__(self, data):
        self.data = data
        self.prices = data['Adj Close'].values
        self.rsi = calculate_rsi(self.prices, 14)
        self.action_space = gym.spaces.Discrete(2)
        self.observation_space = gym.spaces.Box(low=-1, high=1, shape=(1,))
        self.reset()

    def reset(self):
        self.t = 0
        self.done = False
        self.state = self.rsi[self.t]
        return self.state

    def step(self, action):
        reward = 0
        if action == 1:
            if self.prices[self.t + 1] > self.prices[self.t]:
                reward = 1
            else:
                reward = -1
        elif action == 0:
            if self.prices[self.t + 1] < self.prices[self.t]:
                reward = 1
            else:
                reward = -1
        self.t += 1
        self.state = self.rsi[self.t]
        self.done = self.t >= len(self.prices) - 1
        return self.state, reward, self.done, {}

def main():
    # Carga de datos
    data = pd.read_csv('data.csv', index_col='Date')

    # Entorno
    env = RSIEnv(data)

    # Agente
    agent = QLearningAgent(env.observation_space, env.action_space)

    # Entrenamiento
    for episode in range(100):
        state = env.reset()
        while not env.done:
            action = agent.act(state)
            next_state, reward, done, _ = env.step(action)
            agent.learn(state, action, reward, next_state, done)
            state = next_state

    # Evaluación
    env = RSIEnv(data)
    state = env.reset()
    total_reward = 0
    while not env.done:
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        total_reward += reward
        state = next_state

    print(f"Reward total: {total_reward}")

if __name__ == "__main__":
    main()
